# Análisis del riesgo de incumplimiento de los prestatarios

Tu proyecto consiste en preparar un informe para la división de préstamos de un banco. Deberás averiguar si el estado civil y el número de hijos de un cliente tienen un impacto en el incumplimiento de pago de un préstamo. El banco ya tiene algunos datos sobre la solvencia crediticia de los clientes.

Tu informe se tendrá en cuenta al crear una **puntuación de crédito** para un cliente potencial. La **puntuación de crédito** se utiliza para evaluar la capacidad de un prestatario potencial para pagar su préstamo.

## Abre el archivo de datos y mira la información general. 

In [1]:
# Cargar todas las librerías
import pandas as pd
import numpy as np

In [2]:
# Carga los datos
df = pd.read_csv('/datasets/credit_scoring_eng.csv')

## Ejercicio 1. Exploración de datos

**Descripción de los datos**
- `children` - el número de hijos en la familia
- `days_employed` - experiencia laboral en días
- `dob_years` - la edad del cliente en años
- `education` - la educación del cliente
- `education_id` - identificador de educación
- `family_status` - estado civil
- `family_status_id` - identificador de estado civil
- `gender` - género del cliente
- `income_type` - tipo de empleo
- `debt` - ¿había alguna deuda en el pago de un préstamo?
- `total_income` - ingreso mensual
- `purpose` - el propósito de obtener un préstamo

In [3]:
# Vamos a ver cuántas filas y columnas tiene nuestro conjunto de datos

print(f'Cantidad de filas del data frame: {df.shape[0]}')
print(f'Cantidad de columnas del data frame: {df.shape[1]}')

Cantidad de filas del data frame: 21525
Cantidad de columnas del data frame: 12


In [4]:
# vamos a mostrar las primeras filas N

df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


Al analizar la información del dataFrame se observa que existen valores ausentes en las columnas days_employed y total_income. Los valores de las columnas education y purpose requieren estandarización.

Al analizar las primeras 10 filas del dataFrame observamos que en la columna days_employed hay valores negativos, en la columna education hay valores duplicados aparentemente por falta de estandarización en la escritura, hay filas en mayúsculas, minúsculas y una combinación de ambas, el mismo problema se encuentra presente en la columna purpose.

In [5]:
# Obtener información sobre los datos
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,26787.568355
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,16475.450632
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


Existen valores ausentes en las columnas days_employed y total_income.

In [6]:
# Veamos la tabla filtrada con valores ausentes de la primera columna donde faltan datos

print(df[df['days_employed'].isnull()])

       children  days_employed  dob_years            education  education_id  \
12            0            NaN         65  secondary education             1   
26            0            NaN         41  secondary education             1   
29            0            NaN         63  secondary education             1   
41            0            NaN         50  secondary education             1   
55            0            NaN         54  secondary education             1   
...         ...            ...        ...                  ...           ...   
21489         2            NaN         47  Secondary Education             1   
21495         1            NaN         50  secondary education             1   
21497         0            NaN         48    BACHELOR'S DEGREE             0   
21502         1            NaN         42  secondary education             1   
21510         2            NaN         28  secondary education             1   

           family_status  family_status

Los valores ausentes parecen simétricos, una primer suposición sería que son usuarios que nunca han trabajado y por tanto no tienen ingresos o que prefieren no compartir esta información

In [7]:
# Apliquemos múltiples condiciones para filtrar datos y veamos el número de filas en la tabla filtrada.
empty = df[df['days_employed'].isnull() & df['total_income'].isnull()]
print(f'La cantidad de filas con valores nulos es : {empty.shape[0]}' )

La cantidad de filas con valores nulos es : 2174


**Conclusión intermedia**

Observamos que el dataframe empty que contiene los valores ausentes en ambas columnas tiene la misma cantidad de filas que la cantidad de nulos en la columna days_employed, eso nos indica que los valores ausentes son simétricos.

In [8]:
# Vamos a investigar a los clientes que no tienen datos sobre la característica identificada y la columna con los valores ausentes
n = df.shape[0]

print('Porcentaje de valores ausentes:')
print((empty.shape[0] / n)* 100)

Porcentaje de valores ausentes:
10.099883855981417


In [9]:
# Comprobación de la distribución

proyect_without_empty = df[df['days_employed'].notna()]

Generamos el dataframe proyect_without_empty para comparar con el dataframe original y observar si los valores ausentes afectan de manera considerable el dataframe

**Posibles razones por las que hay valores ausentes en los datos**

Como se mencionó anteriormente se consideran 2 opciones para los valores ausentes:

- Usuarios que nunca han trabajado y por tanto no tienen ingresos
- Usuarios que prefieren no compartir esta información

In [10]:
print(100*empty['children'].value_counts(normalize=True))
print()
print(100*df['children'].value_counts(normalize=True))

 0     66.191352
 1     21.849126
 2      9.383625
 3      1.655934
 20     0.413983
 4      0.321987
-1      0.137994
 5      0.045998
Name: children, dtype: float64

 0     65.732869
 1     22.383275
 2      9.547038
 3      1.533101
 20     0.353078
-1      0.218351
 4      0.190476
 5      0.041812
Name: children, dtype: float64


Observamos que los valores ausentes no cambian la distribución de la columna children.

In [11]:
print(100*empty['dob_years'].value_counts(normalize=True))
print()
print(100*df['dob_years'].value_counts(normalize=True))

34    3.173873
40    3.035879
31    2.989880
42    2.989880
35    2.943882
36    2.897884
47    2.713891
41    2.713891
30    2.667893
28    2.621895
57    2.575897
58    2.575897
54    2.529899
38    2.483901
56    2.483901
37    2.437902
52    2.437902
39    2.345906
33    2.345906
50    2.345906
51    2.299908
45    2.299908
49    2.299908
29    2.299908
43    2.299908
46    2.207912
55    2.207912
48    2.115915
53    2.023919
44    2.023919
60    1.793928
61    1.747930
62    1.747930
64    1.701932
32    1.701932
27    1.655934
23    1.655934
26    1.609936
59    1.563937
63    1.333947
25    1.057958
24    0.965961
66    0.919963
65    0.919963
21    0.827967
22    0.781969
67    0.735971
0     0.459982
68    0.413983
69    0.229991
20    0.229991
71    0.229991
70    0.137994
72    0.091996
19    0.045998
73    0.045998
Name: dob_years, dtype: float64

35    2.866434
40    2.829268
41    2.819977
34    2.801394
38    2.778165
42    2.773519
33    2.699187
39    2.662021
31    2

En la columna dob_years tampoco observamos un cambio significativo en la distribución de los datos.

In [12]:
print(100*empty['family_status'].value_counts(normalize=True))
print()
print(100*df['family_status'].value_counts(normalize=True))

married              56.899724
civil partnership    20.331187
unmarried            13.247470
divorced              5.151794
widow / widower       4.369825
Name: family_status, dtype: float64

married              57.514518
civil partnership    19.405343
unmarried            13.068525
divorced              5.551684
widow / widower       4.459930
Name: family_status, dtype: float64


En la columna family_status tampoco observamos un cambio significativo en la distribución de los datos.

In [13]:
print(100*empty['gender'].value_counts(normalize=True))
print()
print(100*df['gender'].value_counts(normalize=True))

F    68.26127
M    31.73873
Name: gender, dtype: float64

F      66.137050
M      33.858304
XNA     0.004646
Name: gender, dtype: float64


En la columna gender tampoco observamos un cambio significativo en la distribución de los datos.

In [14]:
print(100*empty['income_type'].value_counts(normalize=True))
print()
print(100*df['income_type'].value_counts(normalize=True))


employee         50.827967
business         23.367065
retiree          18.997240
civil servant     6.761730
entrepreneur      0.045998
Name: income_type, dtype: float64

employee                       51.656214
business                       23.623693
retiree                        17.914053
civil servant                   6.778165
entrepreneur                    0.009292
unemployed                      0.009292
student                         0.004646
paternity / maternity leave     0.004646
Name: income_type, dtype: float64


En la columna income_type observamos que en el dataframe completo aparecen más categorías pero su porcentaje es tan bajo que no representan un cambio significativo en los datos.

**Conclusiones**

Al comparar ambos dataframes concluimos que los valores ausentes no generan un cambio significativo en la distribución de los datos y por tanto consideramos a los valores ausentes como accidentales.

## Transformación de datos

In [15]:
# Veamos todos los valores en la columna de educación para verificar si será necesario corregir la ortografía y qué habrá que corregir exactamente
df['education'].unique()

array(["bachelor's degree", 'secondary education', 'Secondary Education',
       'SECONDARY EDUCATION', "BACHELOR'S DEGREE", 'some college',
       'primary education', "Bachelor's Degree", 'SOME COLLEGE',
       'Some College', 'PRIMARY EDUCATION', 'Primary Education',
       'Graduate Degree', 'GRADUATE DEGREE', 'graduate degree'],
      dtype=object)

In [16]:
# Arregla los registros si es necesario
df['education'] = df['education'].str.lower()

df['education'] = df['education'].replace('some college', 'college')
df['education'] = df['education'].replace("bachelor's degree", 'bachelors degree')

In [17]:
# Comprobar todos los valores en la columna para asegurarnos de que los hayamos corregido
df['education'].unique()


array(['bachelors degree', 'secondary education', 'college',
       'primary education', 'graduate degree'], dtype=object)

Estandarizamos los datos de la columna education convirtiendolos a minúsculas y cambiando 2 valores.

In [18]:
# Veamos la distribución de los valores en la columna `children`
df['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

Observamos que la columna children tiene 2 valores extraños, el valor -1, ya que no se puede tener un valor negativo de hijos, cambiaremos estos valores usando valor absoluto. El siguiente valor extraño es el valor 20, por la cantidad de filas con este valor simplemente las reemplazaremos por 2 considerando que se trate de un error al capturar los datos

In [19]:
# [arregla los datos según tu decisión]
children = len(df[df['children'] == -1])
print(f'La cantidad de filas con hijos -1 es: {children}')
print()

children = len(df[df['children'] == 20])
print(f'La cantidad de filas con hijos 20 es: {children}')

df['children'] = df['children'].abs()
#Cambiar 20 por 2
df['children'] = df['children'].replace(20,2)

La cantidad de filas con hijos -1 es: 47

La cantidad de filas con hijos 20 es: 76


In [20]:
# Comprobar la columna `children` de nuevo para asegurarnos de que todo está arreglado

df['children'].unique()

#Cambiar 20 por 2 verificando la cantidad de datos

array([1, 0, 3, 2, 4, 5])

En la columna days_employes observamos muchos valores negativos y los valores positivos son sumamente grandes.

Para empezar comprobaremos el porcentaje que representan estos datos erróneos

In [21]:
# Encuentra datos problemáticos en `days_employed`, si existen, y calcula el porcentaje
negative_days_employed = df[df['days_employed'] < 0]
print('Cantidad de valores negativos:')
print(len(negative_days_employed))
print('Porcentaje de valores negativos:')
print((len(negative_days_employed) / len(df))* 100)
print()

not_real_days_employed = df[df['days_employed'] > 0] #Uso 21900 que corresponderia a 60 años
print('Cantidad de valores irreales:')
print(len(not_real_days_employed))
print('Porcentaje de valores irreales:')
print((len(not_real_days_employed) / len(df))* 100)

Cantidad de valores negativos:
15906
Porcentaje de valores negativos:
73.89547038327527

Cantidad de valores irreales:
3445
Porcentaje de valores irreales:
16.004645760743323


Los datos sumamente grandes, al no ser un porcentaje tan grande, los sustituiremos por valores ausentes para que no afecten posteriormente al calculo y a los valores negativos les sacaremos valor absoluto para convertirlos en positivos, ya que posiblemente se deban a un error en la captura de datos.

In [22]:
# Aborda los valores problemáticos, si existen.
df.loc[(df.days_employed > 0), 'days_employed'] = None
df['days_employed'] = df['days_employed'].abs()


In [23]:
# Comprueba el resultado - asegúrate de que esté arreglado
negative_days_employed = df[df['days_employed'] < 0]
print('Cantidad de valores negativos:')
print(len(negative_days_employed))
print('Porcentaje de valores negativos:')
print((len(negative_days_employed) / len(df))* 100)

Cantidad de valores negativos:
0
Porcentaje de valores negativos:
0.0


Ahora revisaremos la edad de los clientes en busca de valores sospechosos.

In [24]:
# Revisa `dob_years` en busca de valores sospechosos y cuenta el porcentaje
dob_years = df['dob_years']
dob_years = dob_years.sort_values(ascending=True)
print(dob_years.unique())

zero_dob_years = df[df['dob_years'] == 0]
print('Cantidad de valores cero:')
print(len(zero_dob_years))
print('Porcentaje de valores cero:')
print((len(zero_dob_years) / len(df))* 100)
print()

[ 0 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41
 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65
 66 67 68 69 70 71 72 73 74 75]
Cantidad de valores cero:
101
Porcentaje de valores cero:
0.4692218350754936



Observamos que hay valores de edad 0, los sustuiremos con la mediana de los valores de edad.

In [25]:
##### Resuelve los problemas en la columna `dob_years`, si existen

median_dob_years = df['dob_years'].median().astype('int64')
df['dob_years'] = df['dob_years'].replace(0,median_dob_years)


In [26]:
# Comprueba el resultado - asegúrate de que esté arreglado
dob_years = df['dob_years']
dob_years = dob_years.sort_values(ascending=True)
print(dob_years.unique())

[19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42
 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66
 67 68 69 70 71 72 73 74 75]


A continuación revisaremos la columna family_status, pero no se identificaron valores conflictivos y no se realizan cambios.

In [27]:
# Veamos los valores de la columna

df['family_status'].unique()

array(['married', 'civil partnership', 'widow / widower', 'divorced',
       'unmarried'], dtype=object)

Analizamos la columna gender y observamos una fila con valor XNA, se decide eliminar dicha fila

In [28]:
# Veamos los valores en la columna
print(df['gender'].unique())

print(df[df['gender'] == 'XNA'])

['F' 'M' 'XNA']
       children  days_employed  dob_years education  education_id  \
10701         0    2358.600502         24   college             2   

           family_status  family_status_id gender income_type  debt  \
10701  civil partnership                 1    XNA    business     0   

       total_income          purpose  
10701     32624.825  buy real estate  


In [29]:
# Aborda los valores problemáticos, si existen

#Eliminar dato na
df.drop(df[df['gender'] == 'XNA'].index, inplace=True)

In [30]:
# Comprueba el resultado - asegúrate de que esté arreglado
df['gender'].unique()


array(['F', 'M'], dtype=object)

A continuación revisaremos la columna income_type, pero no se identificaron valores conflictivos y no se realizan cambios.

In [31]:
# Veamos los valores en la columna
df['income_type'].unique()

array(['employee', 'retiree', 'business', 'civil servant', 'unemployed',
       'entrepreneur', 'student', 'paternity / maternity leave'],
      dtype=object)

Revisamos la columna purpose y observamos muchos valores similares, así que se estandarizan para reducirlo a una cantidad de opciones menor

In [32]:
#Veamos los valores en la columna purpose

df['purpose'].unique()

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

In [33]:
#Se estandarizan los valores de purpose para evitar valores ambiguos o similares

df.loc[df['purpose'].str.contains('house'), 'purpose'] = 'house'
df.loc[df['purpose'].str.contains('car'), 'purpose'] = 'car'
education = ['educated', 'education', 'university']
df.loc[df['purpose'].str.contains('|'.join(education)), 'purpose'] = 'education'
df.loc[df['purpose'].str.contains('wedding'), 'purpose'] = 'wedding'
df.loc[df['purpose'].str.contains('real estate'), 'purpose'] = 'real estate'
df.loc[df['purpose'].str.contains('property'), 'purpose'] = 'property'
df.loc[df['purpose'].str.contains('housing'), 'purpose'] = 'housing'

In [34]:
df['purpose'].unique()

array(['house', 'car', 'education', 'wedding', 'housing', 'real estate',
       'property'], dtype=object)

Observamos que el dataframe cuenta con 247 valores duplicados, al representar un porcentaje bajo de los datos, directamente se eliminan.

In [35]:
# Comprobar los duplicados
print(df.duplicated().sum())
duplicated = df[df.duplicated()]
print(duplicated)

249
       children  days_employed  dob_years            education  education_id  \
829           0            NaN         57  secondary education             1   
1247          0            NaN         54  secondary education             1   
2700          0            NaN         65  secondary education             1   
2849          0            NaN         41  secondary education             1   
3075          0            NaN         62  secondary education             1   
...         ...            ...        ...                  ...           ...   
21281         1            NaN         30     bachelors degree             0   
21321         0            NaN         56  secondary education             1   
21391         0            NaN         52  secondary education             1   
21415         0            NaN         54  secondary education             1   
21502         1            NaN         42  secondary education             1   

           family_status  family_st

In [36]:
# Aborda los duplicados, si existen
df = df.drop_duplicates().reset_index(drop=True)

In [37]:
# Última comprobación para ver si tenemos duplicados
print(df.duplicated().sum())

0


In [38]:
# Comprueba el tamaño del conjunto de datos que tienes ahora, después de haber ejecutado estas primeras manipulaciones
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21275 entries, 0 to 21274
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21275 non-null  int64  
 1   days_employed     15905 non-null  float64
 2   dob_years         21275 non-null  int64  
 3   education         21275 non-null  object 
 4   education_id      21275 non-null  int64  
 5   family_status     21275 non-null  object 
 6   family_status_id  21275 non-null  int64  
 7   gender            21275 non-null  object 
 8   income_type       21275 non-null  object 
 9   debt              21275 non-null  int64  
 10  total_income      19350 non-null  float64
 11  purpose           21275 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 1.9+ MB


Observamos que la cantidad de datos se vio reducida, pero seguimos contando con valores ausentes en las columnas days_employes y total_income.

# Trabajar con valores ausentes

Generamos diccionarios para las columnas family status y education.

In [39]:
# Encuentra los diccionarios

#Generar diccionarios family status y education id

family_dict={
    'married':0,
    'civil partnership':1,
    'widow / widower':2,
    'divorced':3,
    'unmarried':4,
}

education_dict={
    'bachelors degree':0,
    'secondary education':1,
    'college':2,
    'primary education':3,
    'graduate degree':4,
}

### Restaurar valores ausentes en `total_income`

Para trabajar de mejor manera con los datos generaremos la columna age_groups que nos permitirá clasificar y tener una mejor visión de las edades de los usuarios.

In [40]:
# Vamos a escribir una función que calcule la categoría de edad

def age_groups(age):
    if age <= 30:
        return '19-30'
    if 30 < age <= 40:
        return '31-40'
    if 40 < age <= 50:
        return '41-50'
    if 50 < age <= 60:
        return '51-60'
    if 60 < age <= 70:
        return '61-70'
    return '70+'
    

In [41]:
# Prueba si la función funciona bien
print(age_groups(20))
print(age_groups(35))
print(age_groups(42))
print(age_groups(78))

19-30
31-40
41-50
70+


In [42]:
# Crear una nueva columna basada en la función

df['age_group'] = df['dob_years'].apply(age_groups)

In [43]:
# Comprobar cómo los valores en la nueva columna

print(df.head(10))
print()
print(df['age_group'].value_counts())

   children  days_employed  dob_years            education  education_id  \
0         1    8437.673028         42     bachelors degree             0   
1         1    4024.803754         36  secondary education             1   
2         0    5623.422610         33  secondary education             1   
3         3    4124.747207         32  secondary education             1   
4         0            NaN         53  secondary education             1   
5         0     926.185831         27     bachelors degree             0   
6         0    2879.202052         43     bachelors degree             0   
7         0     152.779569         50  secondary education             1   
8         2    6929.865299         35     bachelors degree             0   
9         0    2188.756445         41  secondary education             1   

       family_status  family_status_id gender income_type  debt  total_income  \
0            married                 0      F    employee     0     40620.102   
1

Generamos un nuevo dataframe llamado proyect_without_empty que contendrá los datos no ausentes para calcular media y mediana y usar esos valores para sustituir los valores ausentes

In [44]:
# Crea una tabla sin valores ausentes y muestra algunas de sus filas para asegurarte de que se ve bien
proyect_without_empty = df[df['total_income'].notna()]
print(proyect_without_empty.head(10))

   children  days_employed  dob_years            education  education_id  \
0         1    8437.673028         42     bachelors degree             0   
1         1    4024.803754         36  secondary education             1   
2         0    5623.422610         33  secondary education             1   
3         3    4124.747207         32  secondary education             1   
4         0            NaN         53  secondary education             1   
5         0     926.185831         27     bachelors degree             0   
6         0    2879.202052         43     bachelors degree             0   
7         0     152.779569         50  secondary education             1   
8         2    6929.865299         35     bachelors degree             0   
9         0    2188.756445         41  secondary education             1   

       family_status  family_status_id gender income_type  debt  total_income  \
0            married                 0      F    employee     0     40620.102   
1

Realizamos el cálculo de las medianas y promedio en base al tipo de ingresos y al estatus familiar.

In [45]:
mean_income = proyect_without_empty.groupby('income_type').total_income.mean()
print(mean_income)
print()
median_income = proyect_without_empty.groupby('income_type').total_income.median()
print(median_income)
print()
print()
mean_income = proyect_without_empty.groupby('family_status').total_income.mean()
print(mean_income)
print()
median_income = proyect_without_empty.groupby('family_status').total_income.median()
print(median_income)

income_type
business                       32386.741818
civil servant                  27343.729582
employee                       25820.841683
entrepreneur                   79866.103000
paternity / maternity leave     8612.661000
retiree                        21940.394503
student                        15712.260000
unemployed                     21014.360500
Name: total_income, dtype: float64

income_type
business                       27571.0825
civil servant                  24071.6695
employee                       22815.1035
entrepreneur                   79866.1030
paternity / maternity leave     8612.6610
retiree                        18962.3180
student                        15712.2600
unemployed                     21014.3605
Name: total_income, dtype: float64


family_status
civil partnership    26692.840381
divorced             27189.354550
married              27041.784689
unmarried            26934.069805
widow / widower      22984.208556
Name: total_income, dtype: floa

Generamos los valores promedio de income en base a los grupos de edad que generamos previamente y observaos que el mayor ingreso se encuentra entre los 30 y 50 años, después disminuye, lo cual hace sentido, ya que se entra a las edades de retiro.

In [46]:
# Examina los valores medios de los ingresos en función de los factores que identificaste
mean_income = proyect_without_empty.groupby('age_group').total_income.mean()
print(mean_income)

age_group
19-30    25815.651899
31-40    28376.735148
41-50    28332.806009
51-60    25482.856294
61-70    23245.390243
70+      19575.454327
Name: total_income, dtype: float64


Hacemos el mismo análisis pero para los valores medios.

In [47]:
# Examina los valores medianos de los ingresos en función de los factores que identificaste
median_income = proyect_without_empty.groupby('age_group').total_income.median()
print(median_income)

age_group
19-30    22955.4740
31-40    24825.1865
41-50    24563.6500
51-60    22056.7710
61-70    19705.8550
70+      18611.5935
Name: total_income, dtype: float64


Por la consistencia de los datos, se decide usar los datos clasificados por grupo de edad y específicamente la mediana.

In [48]:
#  Escribe una función que usaremos para completar los valores ausentes

def fill_empty_values(row):
    age_group = row['age_group']
    total_income = row['total_income']
    
    if pd.isna(total_income):
        return median_income[age_group]
    else:
        return total_income

In [49]:
# Comprueba si funciona
df.loc[4,'total_income'] = fill_empty_values(df.loc[4])
print(df.loc[4])

children                              0
days_employed                       NaN
dob_years                            53
education           secondary education
education_id                          1
family_status         civil partnership
family_status_id                      1
gender                                F
income_type                     retiree
debt                                  0
total_income                  25378.572
purpose                         wedding
age_group                         51-60
Name: 4, dtype: object


In [50]:
# Aplícalo a cada fila
df['total_income'] = df.apply(fill_empty_values, axis = 1)

In [51]:
# Comprueba si tenemos algún error
print(df['total_income'].isna().sum())

0


Comprobamos que ya no hay valores ausentes en la columna total_income.

In [52]:
# Comprobar el número de entradas en las columnas

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21275 entries, 0 to 21274
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21275 non-null  int64  
 1   days_employed     15905 non-null  float64
 2   dob_years         21275 non-null  int64  
 3   education         21275 non-null  object 
 4   education_id      21275 non-null  int64  
 5   family_status     21275 non-null  object 
 6   family_status_id  21275 non-null  int64  
 7   gender            21275 non-null  object 
 8   income_type       21275 non-null  object 
 9   debt              21275 non-null  int64  
 10  total_income      21275 non-null  float64
 11  purpose           21275 non-null  object 
 12  age_group         21275 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.1+ MB


###  Restaurar valores en `days_employed`

In [53]:
# Distribución de las medianas de `days_employed` en función de los parámetros identificados
proyect_without_empty = df[df['days_employed'].notna()]

median_days = proyect_without_empty.groupby('age_group').days_employed.median()
print(median_days)

age_group
19-30    1043.576307
31-40    1613.276860
41-50    2054.605931
51-60    2292.866413
61-70    2614.032392
70+      2245.558492
Name: days_employed, dtype: float64


In [54]:
# Distribución de las medias de `days_employed` en función de los parámetros identificados

mean_days = proyect_without_empty.groupby('age_group').days_employed.mean()
print(mean_days)

age_group
19-30    1279.361522
31-40    2090.731584
41-50    2800.921783
51-60    3309.894630
61-70    3868.573391
70+      3933.895823
Name: days_employed, dtype: float64


Se decide utilizar los valores promedio, ya que observamos que tienen una tendencia ascendente, lo cual coincide que entre mayor edad se han trabajado más días.

In [55]:
# Escribamos una función que calcule medias o medianas (dependiendo de tu decisión) según el parámetro identificado
def calc_tendencies (column, tendencie):
    if tendencie == 'mean':
        result = df[column].mean()
    if tendencie == 'median':
        result = df[column].median()
        
    return result

In [56]:
# Comprueba que la función funciona

result = calc_tendencies('days_employed','mean')
print(result)
result = calc_tendencies('days_employed','median')
print(result)

2353.0155808784584
1629.9978622545889


In [57]:
# Aplicar la función al income_type

mean_income = calc_tendencies('total_income','mean')

median_income = calc_tendencies('total_income','median')


In [58]:
# Comprueba si la función funcionó
print('Promedio de ingresos:')
print(mean_income)
print('Mediana de ingresos:')
print(median_income)

Promedio de ingresos:
26476.740397602818
Mediana de ingresos:
23241.043


Se decide trabajar con el promedio de dias trabajados ordenados por grupos de edad para sustituir los valores ausentes.

In [59]:
mean_days = proyect_without_empty.groupby('age_group').days_employed.mean()
print(mean_days)

age_group
19-30    1279.361522
31-40    2090.731584
41-50    2800.921783
51-60    3309.894630
61-70    3868.573391
70+      3933.895823
Name: days_employed, dtype: float64


In [60]:
def fill_empty_values(row):
    age_group = row['age_group']
    days_employed = row['days_employed']
    
    if pd.isna(days_employed):
        return mean_days[age_group]
    else:
        return days_employed

In [61]:
# Reemplazar valores ausentes
df['days_employed'] = df.apply(fill_empty_values, axis = 1)


Finalmente, comprobamos que nuestro data frame ya no contiene valores ausentes.

In [62]:
# Comprueba las entradas en todas las columnas: asegúrate de que hayamos corregido todos los valores ausentes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21275 entries, 0 to 21274
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21275 non-null  int64  
 1   days_employed     21275 non-null  float64
 2   dob_years         21275 non-null  int64  
 3   education         21275 non-null  object 
 4   education_id      21275 non-null  int64  
 5   family_status     21275 non-null  object 
 6   family_status_id  21275 non-null  int64  
 7   gender            21275 non-null  object 
 8   income_type       21275 non-null  object 
 9   debt              21275 non-null  int64  
 10  total_income      21275 non-null  float64
 11  purpose           21275 non-null  object 
 12  age_group         21275 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.1+ MB


## Clasificación de datos

Mostramos las primeras 5 filas para determinar que otras categorías podrían ser útiles

In [63]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437.673028,42,bachelors degree,0,married,0,F,employee,0,40620.102,house,41-50
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car,31-40
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,house,31-40
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,education,31-40
4,0,3309.894630,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,wedding,51-60


Realizaremos 2 categorías más, una para los días trabajados y otra por niveles de ingresos

In [64]:
# Escribamos una función para clasificar los datos en función de temas comunes
df['days_employed'].describe()

count    21275.000000
mean      2558.608007
std       2056.387339
min         24.141633
25%       1016.355551
50%       2212.410477
75%       3340.811396
max      18388.949901
Name: days_employed, dtype: float64

In [65]:
# Crea una columna con las categorías y cuenta los valores en ellas

def days_group(days):
    if days <= 730:
        return 'junior'
    if 730 < days <= 3285:
        return 'mid_senior'
    if 3285 < days:
        return 'senior'
    

In [66]:
df['days_group'] = df['days_employed'].apply(days_group)
print(df.head(10))
print()
print(df['age_group'].value_counts())

   children  days_employed  dob_years            education  education_id  \
0         1    8437.673028         42     bachelors degree             0   
1         1    4024.803754         36  secondary education             1   
2         0    5623.422610         33  secondary education             1   
3         3    4124.747207         32  secondary education             1   
4         0    3309.894630         53  secondary education             1   
5         0     926.185831         27     bachelors degree             0   
6         0    2879.202052         43     bachelors degree             0   
7         0     152.779569         50  secondary education             1   
8         2    6929.865299         35     bachelors degree             0   
9         0    2188.756445         41  secondary education             1   

       family_status  family_status_id gender income_type  debt  total_income  \
0            married                 0      F    employee     0     40620.102   
1

In [67]:
# Escribamos una función para clasificar los datos en función de temas comunes
df['total_income'].describe()

count     21275.000000
mean      26476.740398
std       15751.130908
min        3306.762000
25%       17142.169000
50%       23241.043000
75%       31415.656000
max      362496.645000
Name: total_income, dtype: float64

In [68]:
# Crea una columna con las categorías y cuenta los valores en ellas

def income_groups(income):
    if income <= 19000:
        return 'lower'
    if 19000 < income <= 40000:
        return 'working'
    if 40000 < income <= 150000:
        return 'middle'
    if 150000 < income <= 250000:
        return 'upper_middle'
    if 250000 < income:
        return 'upper'
    

In [69]:
df['income_groups'] = df['total_income'].apply(income_groups)
print(df.head(10))
print()
print(df['income_groups'].value_counts())

   children  days_employed  dob_years            education  education_id  \
0         1    8437.673028         42     bachelors degree             0   
1         1    4024.803754         36  secondary education             1   
2         0    5623.422610         33  secondary education             1   
3         3    4124.747207         32  secondary education             1   
4         0    3309.894630         53  secondary education             1   
5         0     926.185831         27     bachelors degree             0   
6         0    2879.202052         43     bachelors degree             0   
7         0     152.779569         50  secondary education             1   
8         2    6929.865299         35     bachelors degree             0   
9         0    2188.756445         41  secondary education             1   

       family_status  family_status_id gender income_type  debt  total_income  \
0            married                 0      F    employee     0     40620.102   
1

In [70]:
# Contar los valores de cada categoría para ver la distribución
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21275 entries, 0 to 21274
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21275 non-null  int64  
 1   days_employed     21275 non-null  float64
 2   dob_years         21275 non-null  int64  
 3   education         21275 non-null  object 
 4   education_id      21275 non-null  int64  
 5   family_status     21275 non-null  object 
 6   family_status_id  21275 non-null  int64  
 7   gender            21275 non-null  object 
 8   income_type       21275 non-null  object 
 9   debt              21275 non-null  int64  
 10  total_income      21275 non-null  float64
 11  purpose           21275 non-null  object 
 12  age_group         21275 non-null  object 
 13  days_group        21275 non-null  object 
 14  income_groups     21275 non-null  object 
dtypes: float64(2), int64(5), object(8)
memory usage: 2.4+ MB


## Comprobación de las hipótesis


**¿Existe una correlación entre tener hijos y pagar a tiempo?**

In [71]:
child_with_debt = df[df['debt'] == 1]
child_with_debt = child_with_debt.groupby('children')['debt'].count()

100*(child_with_debt/df.groupby('children')['debt'].count())

children
0    7.616725
1    9.205627
2    9.532798
3    8.206687
4    9.756098
5         NaN
Name: debt, dtype: float64

**Conclusión**

Se concluye que los grupos de edad más propensos a tener deuda son los que tienen de 1 a 4 hijos, ya que los que no cuentan con hijos o tienen más de 4 representan un porcentaje inferior.

**¿Existe una correlación entre la situación familiar y el pago a tiempo?**

In [72]:
# Comprueba los datos del estado familiar y los pagos a tiempo

family_with_debt = df[df['debt'] == 1]
family_with_debt = family_with_debt.groupby('family_status')['debt'].count()

# Calcular la tasa de incumplimiento basada en el estado familiar

100*(family_with_debt/df.groupby('family_status')['debt'].count())

family_status
civil partnership    9.401502
divorced             7.118928
married              7.619828
unmarried            9.799714
widow / widower      6.610703
Name: debt, dtype: float64

**Conclusión**

Se concluye que las personas más propensas a tener deuda son las que se encuentra en civil partnership o unmarried.

**¿Existe una correlación entre el nivel de ingresos y el pago a tiempo?**

In [73]:
# Comprueba los datos del nivel de ingresos y los pagos a tiempo

income_with_debt = df[df['debt'] == 1]
income_with_debt = income_with_debt.groupby('income_groups')['debt'].count()

# Calcular la tasa de incumplimiento basada en el nivel de ingresos

100*(income_with_debt/df.groupby('income_groups')['debt'].count())

income_groups
lower            8.108108
middle           6.896552
upper           16.666667
upper_middle     4.545455
working          8.525876
Name: debt, dtype: float64

**Conclusión**

Se concluye que el grupo más propenso a tener deuda son los de mayores ingresos, en este caso superiores a 250,000.

**¿Cómo afecta el propósito del crédito a la tasa de incumplimiento?**

In [74]:
# Consulta los porcentajes de tasa de incumplimiento para cada propósito del crédito y analízalos

purpose_with_debt = df[df['debt'] == 1]
purpose_with_debt = purpose_with_debt.groupby('purpose')['debt'].count()

100*(purpose_with_debt/df.groupby('purpose')['debt'].count())

purpose
car            9.410112
education      9.334006
house          6.698312
housing        6.810982
property       7.524752
real estate    7.603530
wedding        8.069414
Name: debt, dtype: float64

**Conclusión**

Se concluye que las personas cuyo propósito es un carro, educación o una boda son más propenso a tener deuda.

# Conclusión general 


En este proyecto se realizó un trabajo importante de preprocesamiento, donde tuvimos que revisar los valores ausentes, de que manera rellenarlos o eliminarlos, se revisaron valores duplicados, así como revisar cada columna en busca de valores que no tuvieran sentido y determinar si era conveniente reemplazarlos por otros valores o eliminarlos.

Se observó que si únicamente nos basamos en los datos y funciones, sin analizar el contexto del tipo de datos o la información que nos proporcionan, podemos pasar errores por alto o no tratarlos de manera adecuada.

Después de realizar este procesamiento y tener nuestro data frame corregido, se pudieron realizar análisis de los datos para obtener conclusiones y ver patrones, si el propósito afecta, así como el nivel de ingresos, el estatus familiar o la cantidad de hijos, todo esto para apoyar a determinar la calificación crediticia que tiene un posible cliente y tener mejores criterior para determinar la capacidad de pago de un cliente por diversos factores que no sean únicamente sus ingresos.